In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp fastapi.main

<IPython.core.display.Javascript object>

## fastAPI Application Views

```
uvicorn will_it_saturate.fastapi.main:app --reload
```

In [ ]:
# export
import time

from pathlib import Path

from fastapi import FastAPI
from fastapi.responses import FileResponse

<IPython.core.display.Javascript object>

## Time Measuring FileResponse

In [ ]:
# export

import stat
import aiofiles

from aiofiles.os import stat as aio_stat

from starlette.types import Receive, Scope, Send


class MyFileResponse(FileResponse):
    async def __call__(self, scope: Scope, receive: Receive, send: Send) -> None:
        if self.stat_result is None:
            try:
                stat_result = await aio_stat(self.path)
                self.set_stat_headers(stat_result)
            except FileNotFoundError:
                raise RuntimeError(f"File at path {self.path} does not exist.")
            else:
                mode = stat_result.st_mode
                if not stat.S_ISREG(mode):
                    raise RuntimeError(f"File at path {self.path} is not a file.")
        await send(
            {
                "type": "http.response.start",
                "status": self.status_code,
                "headers": self.raw_headers,
            }
        )
        if self.send_header_only:
            await send({"type": "http.response.body", "body": b"", "more_body": False})
        else:
            async with aiofiles.open(self.path, mode="rb") as file:
                start = time.perf_counter()
                more_body = True
                while more_body:
                    chunk = await file.read(self.chunk_size)
                    more_body = len(chunk) == self.chunk_size
                    await send(
                        {
                            "type": "http.response.body",
                            "body": chunk,
                            "more_body": more_body,
                        }
                    )
                elapsed = time.perf_counter() - start
                print(f"elapsed: {elapsed} for {self.path}")

        if self.background is not None:
            await self.background()

<IPython.core.display.Javascript object>

## FastAPI File Serving Views

In [ ]:
# export

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/pathinfo/{base_dir}")
def get_file_urls(base_dir: str):
    pass


@app.get("/{base}/{path}/{num}")
async def serve_file(base: str, path: str, num: int):
    file_path = str(Path(base) / path / str(num))
    return MyFileResponse(file_path)

<IPython.core.display.Javascript object>

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_index.ipynb.
Converted 01_host.ipynb.
Converted 02_file.ipynb.
Converted 03_registry.ipynb.
Converted 04_epochs.ipynb.
Converted 10_servers.ipynb.
Converted 11_views_for_fastapi_server.ipynb.
Converted 12_views_for_django_server.ipynb.
Converted 15_servers_started_locally.ipynb.
Converted 16_servers_started_via_docker.ipynb.
Converted 20_clients.ipynb.
Converted 21_benchmark_client_implementations.ipynb.
Converted 30_control_server.ipynb.
Converted 31_control_client.ipynb.
Converted 32_control_cli.ipynb.
Converted 40_results.ipynb.
Converted 41_repositories.ipynb.
Converted 42_sqlite_repository.ipynb.
Converted 50_benchmark_without_benchmark.ipynb.
Converted 51_benchmark_remote_control_server.ipynb.
Converted 52_benchmark_from_config.ipynb.
Converted 52_generate_files_for_benchmark.ipynb.
Converted 60_legacy.ipynb.
Converted 61_run_benchmark.ipynb.


<IPython.core.display.Javascript object>